In [1]:
%matplotlib inline 
# plots graphs within the notebook
%config InlineBackend.figure_format='svg' # not sure what this does, may be default images to svg format

from IPython.display import Image

from IPython.core.display import HTML
def header(text):
    raw_html = '<h4>' + str(text) + '</h4>'
    return raw_html

def box(text):
    raw_html = '<div style="border:1px dotted black;padding:2em;">'+str(text)+'</div>'
    return HTML(raw_html)

def nobox(text):
    raw_html = '<p>'+str(text)+'</p>'
    return HTML(raw_html)

def addContent(raw_html):
    global htmlContent
    htmlContent += raw_html

### Library for thermodynamic properties

Defines thermodynamic properties of air and water at 1 atm. Air properties are tabled between $-150\text{$^\circ$C}$ and $400\text{$^\circ$C}$, water between $274\text{K}$ and $373\text{K}$, Argon between $100\text{K}$ and $700\text{K}$ and Krypton between $150\text{K}$ and $750\text{K}$
<p class='alert alert-danger'>
<b>Input must be in Kelvin</b>
</p>
Use the scipy functions <FONT FACE="courier" style="color:blue">C2K </FONT> and <FONT FACE="courier" style="color:blue">F2K </FONT> to convert temperatures to Kelvin. Thermodynamic properties are linearly interpolated from the two nearest temperature states.

In [2]:
%%file thermodynamics.py
import numpy as np
import scipy
import scipy.optimize
from scipy.constants.constants import C2K
from scipy.constants.constants import K2C
from scipy.constants.constants import F2K
from scipy.constants.constants import K2F
import scipy.constants as sc

def interpolate_table(target,index,xquantity,yquantity):
    return yquantity[index] + \
                (yquantity[index+1]-yquantity[index])* \
                (target-xquantity[index])/(xquantity[index+1]-xquantity[index])
        
class Fluid(object):
    """ Compute thermodynamics properties of air between -150 C and 400 C, 
        water between 274K and 373K, argon between 100 and 700K and
        krypton between 150 and 700 K under 1 atm. Argon, krypton and water were obtained 
        through http://webbook.nist.gov/chemistry/fluid/
        Declare your fluid as fluid=Fluid('name') where
        name is air, water, argon or krypton
        Then run fluid.get_properties(T) where T is your temperature
        Note that the temperature must be in Kelvin
        More fluids to be added in the future"""
    def __init__(self,name):
        self.name = name
        
    def get_properties(self,T_o):
        self.T = T_o
        if self.name == 'water':
            if T_o < 274 or T_o > 373:
                print("Temperature is out of bounds for liquid water")
                return 
            Ttab,ptab,rhotab,Cptab,mutab,ktab = \
            np.genfromtxt('Tables/water1atm.csv', delimiter=',', skip_header = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Cptab *= 1e3
            nutab = mutab/rhotab 
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            dTtab = Ttab[1] - Ttab[0]
            # compute beta from -rho(d rho/dT)
            betatab = -(1./rhotab)*np.gradient(rhotab)/dTtab
            i = int((T_o-Ttab[0])/dTtab)
            if (i == Ntab - 1):
                i == Ntab - 2
        elif self.name == 'argon':
            if T_o < 100 or T_o > 700:
                print("Temperature is out of bounds for argon")
                return 
            Ttab,ptab,rhotab,Cptab,mutab,ktab = \
            np.loadtxt('Tables/Argon1atm.csv', delimiter=',', skiprows = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Cptab *= 1e3
            nutab = mutab/rhotab 
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            dTtab = Ttab[1] - Ttab[0]
            # compute beta from -rho(d rho/dT)
            betatab = -(1./rhotab)*np.gradient(rhotab)/dTtab
            i = int((T_o-Ttab[0])/dTtab)
            if (i == Ntab - 1):
                i == Ntab - 2
        elif self.name == 'krypton':
            if T_o < 150 or T_o > 740:
                print("Temperature is out of bounds for krypton")
                return 
            Ttab,ptab,rhotab,Cptab,mutab,ktab = \
            np.loadtxt('Tables/Krypton1atm.csv', delimiter=',', skiprows = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Cptab *= 1e3
            nutab = mutab/rhotab 
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            dTtab = Ttab[1] - Ttab[0]
            # compute beta from -rho(d rho/dT)
            betatab = -(1./rhotab)*np.gradient(rhotab)/dTtab
            i = int((T_o-Ttab[0])/dTtab)
            if (i == Ntab - 1):
                i == Ntab - 2
        elif self.name == 'air':
            if T_o < C2K(-150.) or T_o > C2K(400.):
                print("Temperature is out of bounds of the table for air")
                return
            Ttab,rhotab,Cptab,ktab,nutab,betatab,Prtab = \
            np.genfromtxt('Tables/air1atm.csv', delimiter=',', skip_header = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Ttab = C2K(Ttab)
            Cptab *= 1e3
            nutab *= 1e-6
            mutab = rhotab*nutab
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            i = 0
            while (Ttab[i] < T_o) and (i<Ntab):
                i += 1
            i -=1
            if (i == Ntab - 1):
                i = Ntab - 2
            
        else:
            print("warning, no table available for", self.name)
            return
        
        self.rho = interpolate_table(T_o,i,Ttab,rhotab)
        self.Cp = interpolate_table(T_o,i,Ttab,Cptab)
        self.mu = interpolate_table(T_o,i,Ttab,mutab)
        self.k = interpolate_table(T_o,i,Ttab,ktab)
        self.nu = interpolate_table(T_o,i,Ttab,nutab)
        self.alpha = interpolate_table(T_o,i,Ttab,alphatab)
        self.Pr = interpolate_table(T_o,i,Ttab,Prtab)
        if (self.name == 'air'):
            self.beta = 1./T_o
        else:
            self.beta = interpolate_table(T_o,i,Ttab,betatab)
        

Overwriting thermodynamics.py


In [3]:
import thermodynamics as thermo
import numpy as np
from scipy.constants.constants import C2K
from scipy.constants.constants import K2C
from scipy.constants.constants import F2K
from scipy.constants.constants import K2F

fluid = thermo.Fluid('argon')
print(fluid.name)
#array = np.genfromtxt('Tables/Argon1atm.csv', delimiter=',', skip_header = 1, unpack=True, dtype=float)
#print(array[0,:])
fluid.get_properties(C2K(5.))
print(fluid.k)

argon
0.01662565


### Library of thermal resistances

In [4]:
%%file HT_thermal_resistance.py
### definition of thermal resistance ###

import numpy as np
import math
import scipy.constants as sc

class Resistance(object):
    """ Defines thermal resistances for conduction, convection and radiation heat transfer. 
        First define the mode when defining your object
        Second use self.conduction, self.convection or self.radiation to calculate your resistance """
    def __init__(self,name):
        self.name = name
    def conduction(self,geo,k,r_a,r_b,A):
        self.geometry = geo
        if self.geometry == 'plane':
            self.R = r_a/(k*A)
        elif self.geometry == 'cylindrical':
            self.R = np.log(r_b/r_a)/(2.*math.pi*L*k)
        elif self.geometry == 'spherical':
            self.R = (1./r_a-1./r_b)/(4.*math.pi*k)
        else :
            print("geometry is not plane, cylindrical or spherical, cannot compute")
    def convection(self,h,A):
        self.R = 1./(h*A)
    def radiation(eps,T_s,T_infty,A):
        self.R = 1./(eps*sc.sigma*(T_s+T_infty)*(T_s**2+T_infty**2)*A)
        
### summation of thermal resistance (R is a vector) ###
def serial_sum(R,nori,nend):
    sum = 0.
    for i in range(nori,nend+1):
        sum += R[i].R
    return sum

def parallel_sum(R,nori,nend):
    sum = 0.
    for i in range(nori,nend+1):
        sum += 1./R[i].R
    return 1./sum

Overwriting HT_thermal_resistance.py


In [5]:
from HT_thermal_resistance import Resistance,serial_sum,parallel_sum

Rth = []
Rth.append(Resistance('Rcond1')) 
Rth[0].conduction('plane',0.01,1.,0.,1.)
Rth.append(Resistance('Rcond1')) 
Rth[1].conduction('plane',10.,1.,0.,1.)
print(Rth[1].name)


print(parallel_sum(Rth,0,1))
    
#R_total = serial_sum(Rth[:].R)
#print(R_total)

Rcond1
0.0999000999001


### Library of Nu correlations for external flow around a pipe

In [6]:
%%file HT_external_convection_cylinder.py
def Nu_Hilbert(Re,Pr):
    if (Re < 0.4) or (Re > 400000.):
        Nu = 0.
    else:
        if (Re < 4.):
            C = 0.989
            m = 0.33
        elif (Re < 40.):
            C = 0.911
            m = 0.385
        elif (Re < 4000.):
            C = 0.683
            m = 0.466
        elif (Re < 40000.):
            C = 0.193
            m = 0.618
        else:
            C = 0.027
            m = 0.805
        Nu = C*Re**m*Pr**(1./3.)
    return Nu

def Nu_Churchill_Bernstein(Re,Pr):
    
    if (Re*Pr < 0.2):
        Nu = 0.
    
    else:
        Nu = 0.3+(0.62*Re**(0.5)*Pr**(1./3.)) \
          /(1.+(0.4/Pr)**(2./3.))**(1./4.) \
        *(1.+(Re/282000.)**(5./8.))**(4./5.)
    return Nu

def Nu_Zukauskas(Re,Pr,Pr_s):
    if (Pr <= 10):
        n = 0.37
    else:
        n = 0.36
    inbound = True
    if (Re < 1.) and (Re > 1.e6):
        Nu = 0.
    else:
        if (Re < 40.):
            C = 0.75
            m = 0.4
        elif (Re < 1000.):
            C = 0.51
            m = 0.5
        elif (Re < 2.e5):
            C = 0.26
            m = 0.6
        else:
            C = 0.076
            m = 0.7
        Nu = C*Re**m*Pr**n*(Pr/Pr_s)**(1./4.)
    return Nu

Overwriting HT_external_convection_cylinder.py


### Library of Nu correlations and functions for internal flow in pipes

In [7]:
%%file HT_internal_convection.py
import numpy as np
import scipy
import scipy.optimize

def linear_interpolation(x_t,x_1,x_2,y_1,y_2):
    return y_1+(y_2-y_1)*(x_t-x_1)/(x_2-x_1)

def pressure_drop_pipe(f,L,D,rho,u_m):
    return f*(L/D)*(rho*u_m**2)/2.

def f_pipe_laminar(Re_D):
    return 64./Re_D

def f_pipe_colebrook(Re_D,eps):
    Re = Re_D
    e = eps
     
    f_0 = (0.790*np.log(Re)- 1.64)**(-2.)
    if (e > 0.):
        f_1 = 1./(-2.0*np.log10(e/3.71))**2
    else:
        f_1 = f_0
    f_guess = np.max(f_0,f_1)
    #f_guess = 0.04
    def f_tmp(x):
        y = (-2*np.log10((2.51/(Re*np.sqrt(x))) + (e/(3.71))) - 1.0/np.sqrt(x))
        return y
    y = scipy.optimize.fsolve(f_tmp, f_guess)
    return y
def log_mean_temperature(T_s,T_o,T_i):
    if (T_s < min(T_o,T_i)):
        DT_o = T_o-T_s
        DT_i = T_i-T_s
    elif (T_s > max(T_o,T_i)):
        DT_o = T_s-T_o
        DT_i = T_s-T_i
    return (DT_o-DT_i)/np.log(DT_o/DT_i)

def T_mx_Ts_constant(T_s,T_mi,P,mdot,Cp,hbar,x):
    return T_s-(T_s-T_mi)*np.exp(-P*x*hbar/(mdot*Cp))

def T_mo_T_infty(T_infty,T_mi,P,L,mdot,Cp,R_tot):
    return T_infty-(Tinfty-T_mi)*np.exp(-1/(mdot*Cp*Rtot))

def hbar_laminar_isothermal(k,D):
    return 3.66*k/D

def hbar_laminar_isoflux(k,D):
    return 4.36*k/D

def Nu_turbulent_Dittus_Boelter(Re,Pr,mode):
    if (mode == 'heating'):
        n = 0.4
    elif (mode == 'cooling'):
        n = 0.3
    return 0.023*Re**(4./5.)*Pr**n

def Nu_turbulent_Sieder_Tate(Re,Pr,mu,mu_s):
    return 0.027*Re**(4./5.)*Pr*(1./3.)*(mu/mu_s)**0.14

def Nu_turbulent_Gnielinski(Re,Pr,f):
    return (f/8.)*(Re-1000.)*Pr/(1+12.7*(f/8.)**0.5*(Pr**(2./3.)-1.))

def Nu_turbulent_Skupinski(Re,Pr):
    return 4.82+0.0185*(Re*Pr)**0.827

def Nu_turbulent_Seban(Re,Pr):
    return 5.0+0.025*(Re*Pr)**0.8

Overwriting HT_internal_convection.py


### Library for natural convection around cylinders

In [8]:
%%file HT_natural_convection_cylinder.py
import numpy as np
import scipy
import scipy.optimize

def Gr(g,beta,DT,D,nu):
    return (g*beta*DT*D**3)/(nu**2)

def Ra(g,beta,DT,D,nu,alpha):
    return (g*beta*DT*D**3)/(nu*alpha)


def Nu_Morgan(Ra):
    if (Ra <= 1e-2):
        C=0.675
        n=0.058
    elif (Ra <= 1e2):
        C=1.02
        n=0.148
    elif (Ra <= 1e4):
        C=0.85
        n=0.188
    elif (Ra <= 1e7):
        C=0.480
        n=0.250
    elif (Ra <= 1e12):
        C=0.125
        n=0.333
    return C*Ra**n

def Nu_Churchill_Chu(Ra,Pr):
    return (0.60+(0.387*Ra**(1./6.))/(1.+(0.559/Pr)**(9./16.))**(8./27.))**2 


Overwriting HT_natural_convection_cylinder.py


### Library of natural convection in enclosure

In [9]:
%%file HT_natural_convection_enclosure.py
import numpy as np
import scipy
import scipy.optimize

def Gr(g,beta,DT,L,nu):
    return (g*beta*DT*L**3)/(nu**2)

def Ra(g,beta,DT,L,nu,alpha):
    return (g*beta*DT*L**3)/(nu*alpha)

def Nu_vertical_enclosure(Ra,Pr,H,L):
    if (H/L) < 2.:
        if Ra*Pr/(0.2+Pr)> 1.e3:
            Nu = 0.18*(Pr/(0.2+Pr)*Ra)**0.29
        else:
            print('Ra is too low for this correlation')
            Nu = np.inf
    elif H/L < 10:
        if Ra < 1e10:
            Nu = 0.22*(Pr/(0.2+Pr)*Ra)**0.28*(H/L)**(-0.25)
        else:
            print('Ra is too high for this correlation')
            Nu = np.inf
    elif Ra < 1e4:
        print('Ra is too low for this correlation')
        Nu = np.inf
    elif Ra < 1e7:
        if Pr > 0.6 and Pr < 2e4:
            print('ok')
            Nu =0.42*Ra**0.25*Pr**0.012*(H/L)**(-0.3)
        else :
            print('Pr is out of bounds for this correlation')
            Nu = np.inf
    elif Ra < 1e9:
        if Pr > 0.6 and Pr < 20.:
            Nu =0.46*Ra**(1./3.)
        else :
            print('Pr is out of bounds for this correlation')
            Nu = np.inf
    else:
        print('Ra is too high, got nothing for you')
        Nu = np.inf
    return Nu
            

Overwriting HT_natural_convection_enclosure.py
